In [1]:
#Grover's Algorithm
#Samwel Sekwao
#05/21/2021

#Implementation of the paper by Figgatt et. al

In [2]:
import random
import numpy as np
import cirq
import statistics
from statistics import mode
from cirq import H, X, CNOT, S,measure

In [40]:
n = 3 #Number of qubits
#Number of iterations required for general n = pi*sqrt(2**n)/4 - 1/2
T = int(np.pi*np.sqrt(2**3)/4 - 0.5)
print('Number of iterations required = %d\n'%(T))

qbs = cirq.LineQubit.range(n) #Qubits
qc = cirq.Circuit()

#Intial state |s> = |+++>
#Apply H gate to all qubits
for i in range(n):
    qc.append(H(qbs[i]))
    
print("Circuit:")
print(qc)

Number of iterations required = 1

Circuit:
0: ───H───

1: ───H───

2: ───H───


In [41]:
#Oracle: Uw I - 2|w><w|
#Oracle will be for the states |w1> = |110> and |w2> = |101>
#Uw|w1> = -|w1>
#Uw|w2> = -|w2>
#Add Oracle implemented in the paper
for i in range(1,n):
    qc.append(cirq.Z(qbs[0]).controlled_by(qbs[i]))
    
print("Circuit:")
print(qc)

Circuit:
0: ───H───Z───Z───
          │   │
1: ───H───@───┼───
              │
2: ───H───────@───


In [42]:
#Amplification: Us = 2|s><s| - I
#|s> = |+++>
#Add amplification implemented in the paper
for i in range(n):
    qc.append([H(qbs[i]),X(qbs[i])])
    
qc.append(cirq.Z(qbs[0]).controlled_by(qbs[1],qbs[2]))

print("Circuit:")
print(qc)

Circuit:
              ┌──┐
0: ───H───Z────Z─────H───X───Z───
          │    │             │
1: ───H───@────┼H────X───────@───
               │             │
2: ───H────────@─────H───X───@───
              └──┘


In [43]:
#Add X, and H to each qubit
for i in range(n):
    qc.append([X(qbs[i]),H(qbs[i])])

print("Circuit:")
print(qc)

Circuit:
              ┌──┐
0: ───H───Z────Z─────H───X───Z───X───H───
          │    │             │
1: ───H───@────┼H────X───────@───X───H───
               │             │
2: ───H────────@─────H───X───@───X───H───
              └──┘


In [44]:
#Final state vector before measurement
#Final state vector should be (|110> + |101>)/sqrt(2)
simulator = cirq.Simulator()
result = simulator.simulate(qc, qubit_order=[qbs[0], qbs[1],qbs[2]])
print(result)

measurements: (no measurements)
output vector: -0.707|101⟩ - 0.707|110⟩


In [45]:
#Add measurement
qc.append(measure(qbs[0],qbs[1],qbs[2],key='Result'))

print("Circuit:")
print(qc)

Circuit:
              ┌──┐
0: ───H───Z────Z─────H───X───Z───X───H───M('Result')───
          │    │             │           │
1: ───H───@────┼H────X───────@───X───H───M─────────────
               │             │           │
2: ───H────────@─────H───X───@───X───H───M─────────────
              └──┘


In [46]:
#Simulate the circuit
Order_List = [qbs[i] for i in range(n)] #Order list for the results
simulator = cirq.Simulator()
results = simulator.simulate(qc, qubit_order=Order_List)
print(results)

#Just 1 query of the oracle will reveal 5 = |101> or 6 = |110> 
samples=simulator.run(qc,repetitions=50000)
print(samples.histogram(key='Result'))

measurements: Result=110
output vector: -1|110⟩
Counter({6: 25072, 5: 24928})
